In [1]:
# Functional Package Import
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import math
import netCDF4
import numpy.ma as ma
from pylab import *
import csv
import random
import copy
from mpl_toolkits.basemap import Basemap
from matplotlib.ticker import LogLocator, ScalarFormatter

# Data file reader

# .xlsx reader
def readexcel(file_name):
    
    data = pd.read_excel(file_name)
    train_data = np.array(data)  # np.ndarray()
    excel_list = train_data.tolist()  # list
    
    return excel_list

# .csv reader
def csvimport(file_name):
    
    smiles = []
    with open(file_name,'r',encoding='UTF-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:  
            smiles.append(row)
    smiles[0][0] = 'No.'
    
    return(smiles) 

def csvimport1(file_name):
    
    smiles = []
    with open(file_name) as csvfile:
        csv_reader = csv.reader(csvfile)  
        for row in csv_reader: 
            smiles.append(row)
    smiles[0][0] = 'No.'
    
    return(smiles) 

r"""
   The functions are defined for data import from .csv file (csvimport and csvimport1) 
   and .xlsx file (readexcel)
"""

# Seawater Density according to depth, salinity, and temperature
def water_density(t,S):
    
    c1 = 9.999e02
    c2 = 2.034e-02
    c3 = -6.162e-03
    c4 = 2.261e-05
    c5 = -4.657e-08
    b1 = 8.020e02
    b2 = -2.001
    b3 = 1.677e-02
    b4 = -3.060e-05
    b5 = -1.613e-05
    
    den_f = c1 + c2*t + c3*t**2 + c4*t**3 + c5*t**4 + b1*S + b2*S*t + b3*S*t**2 + b4*S*t**3 + b5*(S**2)*(t**2) 
    value = den_f/1000
    
    return(value)

    r"""
    Calculates seawater density at given temperature and salinity
    using Eq. (8) given by Sharqawy et. al [1]. Values at temperature higher
    than the normal boiling temperature are calculated at the saturation
    pressure.
 
    Parameters
    ----------
    t : float
        Temperature must be in Celsius for this emperical equation to work
 
    S : float
        Salinity must be expressed in kg of salt per kg of solution (ppt).
 
    Returns
    -------
    value, the density of water/seawater in [kg/m3]
 
    Notes
    -----
    T must be in C, and S in g of salt per kg of phase, or ppt (parts per thousand)
    VALIDITY: 0 < T < 180 C; 0 < S < 0.16 kg/kg;
    ACCURACY: 0.1 %
 
    References
    ----------
    [1] Sharqawy M. H., Lienhard J. H., and Zubair, S. M., Desalination and
        Water Treatment, 2010.
    
    
    location = [['MLH','36.8N','122.5W'],['SK1','23.2N','127.3E'],['SK2','12.1N','141.8E'],['SK3','14.8N','145.8E']
            ,['SY1','13.5N','87.3E'],['SY2','0.1N','87.3E'],['SY3','13.5S','87.3E'],['DP1','33.7N','40.5W']
            ,['DP2','34.4N','36.2W'],['EGIV','78.8N','2.8E'],['N5','79.9N','3.1W'],['HGIV','79.1N','4.2W']
            ,['HGIX','79.1N','2.8W']]
    """

def main():
    # Data import
    date = '20220628'
    url = './Data/' + date + '_rtofs_glo_3dz_n024_daily_3ztio.nc'
    file = netCDF4.Dataset(url)
    lat  = file.variables['Latitude'][:]
    lon  = file.variables['Longitude'][:]
    # Extract temperature data from given location
    temp = file.variables['temperature'][0,:,:,:]
    file.close()
    url = './Data/' + date + '_rtofs_glo_3dz_n024_daily_3stio.nc'
    file = netCDF4.Dataset(url)
    # Extract salinty data from given location
    sat = file.variables['salinity'][0,:,:,:]
    file.close()
    lon[3297] = lon[3296] + lon[3296] - lon[3295]

    new_layer_1 = water_density(temp,sat/1000)
    xmax = np.amax(new_layer_1,axis=0)
    xmin = np.amin(new_layer_1,axis=0)
    final_density_diff = xmax - xmin

    plt.figure(figsize=(10,5),dpi=400)
    n = Basemap(projection='mill',lat_ts=10,
      llcrnrlon=74,urcrnrlon=434,llcrnrlat=lat.min(),urcrnrlat=lat.max(), resolution='c')
    x,y = n(lon,lat)
    cs = n.pcolormesh(x,y,final_density_diff[1:,1:],shading='flat',cmap=plt.cm.jet)
    n.drawcoastlines()
    n.fillcontinents()
    n.drawmapboundary()
    n.drawparallels(np.arange(-90.,120.,30.), labels=[1,0,0,0])
    n.drawmeridians(np.arange(-180.,180.,60.), labels=[0,0,0,1])
    cb = plt.colorbar()
    cb.set_label('Density increment from surface to seafloor (g cm\u207B\u00B3)',rotation=90)
    plt.title('(C) Vertical Density Increament in the Global Oceans (June 2022)',fontsize=15)

    position_lon = [-122.3+360, 127.3, 141.8, 145.8, 87.3, 87.3, 87.3, -40.5+360, -36.2+360, 2.8+360, -3.1+360, -4.2+360, -2.8+360]
    position_lat = [36.8, 23.2, 12.1, 14.8, 13.5, 0.1, -13.5, 33.7, 34.4, 78.8, 79.9, 79.1, 79.1]

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="r")

    position_lon = [50+360, 10+360, -10+360, 5+360, 4+360, 3+360,142]
    position_lat = [-40, 40, 48, 78.8, 79.9, 79.1,11]    

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="yellow")

    plt.savefig('./Output/Fig5_C.jpg')

    # Data import
    date = '20211208'
    url = './Data/' + date + '_rtofs_glo_3dz_n024_daily_3ztio.nc'
    file = netCDF4.Dataset(url)
    lat  = file.variables['Latitude'][:]
    lon  = file.variables['Longitude'][:]
    temp = file.variables['temperature'][0,:,:,:]
    file.close()
    # Data import
    url = './Data/' + date + '_rtofs_glo_3dz_n024_daily_3stio.nc'
    file = netCDF4.Dataset(url)
    # Extract data from given location
    sat = file.variables['salinity'][0,:,:,:]
    file.close()
    lon[3297] = lon[3296] + lon[3296] - lon[3295]

    new_layer_1 = water_density(temp,sat/1000)
    xmax = np.amax(new_layer_1,axis=0)
    xmin = np.amin(new_layer_1,axis=0)
    final_density_diff = xmax - xmin

    plt.figure(figsize=(10,5),dpi=400)
    n = Basemap(projection='mill',lat_ts=10,
      llcrnrlon=74,urcrnrlon=434,llcrnrlat=lat.min(),urcrnrlat=lat.max(), resolution='c')
    x,y = n(lon,lat)
    cs = n.pcolormesh(x,y,final_density_diff[1:,1:],shading='flat',cmap=plt.cm.jet)
    n.drawcoastlines()
    n.fillcontinents()
    n.drawmapboundary()
    n.drawparallels(np.arange(-90.,120.,30.), labels=[1,0,0,0])
    n.drawmeridians(np.arange(-180.,180.,60.), labels=[0,0,0,1])
    cb = plt.colorbar()
    cb.set_label('Density increment from surface to seafloor (g cm\u207B\u00B3)',rotation=90)
    plt.title('(E) Vertical Density Increament in the Global Oceans (December 2021)',fontsize=15)

    position_lon = [-122.3+360, 127.3, 141.8, 145.8, 87.3, 87.3, 87.3, -40.5+360, -36.2+360, 2.8+360, -3.1+360, -4.2+360, -2.8+360]
    position_lat = [36.8, 23.2, 12.1, 14.8, 13.5, 0.1, -13.5, 33.7, 34.4, 78.8, 79.9, 79.1, 79.1]

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="r")

    position_lon = [50+360, 10+360, -10+360, 5+360, 4+360, 3+360,142]
    position_lat = [-40, 40, 48, 78.8, 79.9, 79.1,11]    

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="yellow")

    plt.savefig('./Output/Fig5_E.jpg')

    # Surface chlorophyll-a database import
    chl_a_file_name = "./Data/Chl_a_surface concentration_winter.csv"
    chl_a_file = csvimport(chl_a_file_name)
    chl_a_file = np.array(chl_a_file)
    Lon_bins = chl_a_file[0,1:]
    Lat_bins = chl_a_file[1:,0]
    Lat_bins = Lat_bins.astype('float')
    Lon_bins = Lon_bins.astype('float')
    dataset = chl_a_file[1:,1:].astype('float')

    Lon = np.hstack((Lon_bins[1800:3600]-360,Lon_bins[0:1800]))
    lon = np.tile(Lon,(1800,1))
    lat = np.transpose(np.tile(Lat_bins,(3600,1)))
    mask = (dataset == 99999.)
    dataset[mask] = np.nan
    chl_a = np.log10(dataset)
    chl_a_tran = np.hstack((chl_a[:,1800:3600],chl_a[:,0:1800]))
    plt.figure(figsize=(10,5),dpi=400)
    n = Basemap(projection='mill',lat_ts=10,
      llcrnrlon=-360,urcrnrlon=0,llcrnrlat=lat.min(),urcrnrlat=lat.max(), resolution='c')
    x,y = n(lon,lat)
    cs = n.pcolormesh(x,y,chl_a_tran[1:,1:],shading='flat',cmap=plt.cm.jet)
    n.drawcoastlines()
    n.fillcontinents()
    n.drawmapboundary()
    n.drawparallels(np.arange(-90.,120.,30.), labels=[1,0,0,0])
    n.drawmeridians(np.arange(-180.,180.,60.), labels=[0,0,0,1])
    cb = plt.colorbar()
    ticks = ['0.01','0.03','0.1','0.3','1.0','3.2','10','31']
    cb.set_ticklabels(ticks)
    plt.title('(D) Global Chlorophyll Concentration (December 2021)',fontsize=15)
    cb.set_label('Chlorophyll Concentration (mg m\u207B\u00B3)', rotation=90,x = 100)
    position_lon = [-122.3, 127.3-360, 141.8-360, 145.8-360, 87.3-360, 87.3-360, 87.3-360, -40.5, -36.2, 2.8-360, -3.1, -4.2, -2.8]
    position_lat = [36.8, 23.2, 12.1, 14.8, 13.5, 0.1, -13.5, 33.7, 34.4, 78.8, 79.9, 79.1, 79.1]

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="r")

    position_lon = [50-360, 10-360, -10, 5-360, 4-360, 3-360,142-360]
    position_lat = [-40, 40, 48, 78.8, 79.9, 79.1,11]    

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="yellow")

    plt.savefig('./Output/Fig5_D.jpg')

    # Surface chlorophyll-a database import
    chl_a_file_name = "./Data/Chl_a_surface_concentration_summer.csv"
    chl_a_file = csvimport(chl_a_file_name)
    chl_a_file = np.array(chl_a_file)
    chl_a_file[0,0] = 99999.0
    dataset = chl_a_file[1:,1:].astype('float')

    mask = (dataset > 100.)
    dataset[mask] = np.nan
    chl_a = np.log10(dataset)
    chl_a_tran = np.hstack((chl_a[:,1800:3600],chl_a[:,0:1800]))

    plt.figure(figsize=(10,5),dpi=400)
    n = Basemap(projection='mill',lat_ts=10,
      llcrnrlon=-360,urcrnrlon=0,llcrnrlat=lat.min(),urcrnrlat=lat.max(), resolution='c')
    x,y = n(lon,lat)
    cs = n.pcolormesh(x,y,chl_a_tran,shading='flat',cmap=plt.cm.jet)
    n.drawcoastlines()
    n.fillcontinents()
    n.drawmapboundary()
    n.drawparallels(np.arange(-90.,120.,30.), labels=[1,0,0,0])
    n.drawmeridians(np.arange(-180.,180.,60.), labels=[0,0,0,1])
    cb = plt.colorbar()
    ticks = ['0.01','0.03','0.1','0.3','1.0','3.2','10','31']
    cb.set_ticklabels(ticks)
    plt.title('(B) Global Chlorophyll Concentration (June 2022)',fontsize=15)
    cb.set_label('Chlorophyll Concentration (mg m\u207B\u00B3)', rotation=90,x = 100)
    position_lon = [-122.3, 127.3-360, 141.8-360, 145.8-360, 87.3-360, 87.3-360, 87.3-360, -40.5, -36.2, 2.8-360, -3.1, -4.2, -2.8]
    position_lat = [36.8, 23.2, 12.1, 14.8, 13.5, 0.1, -13.5, 33.7, 34.4, 78.8, 79.9, 79.1, 79.1]

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="r")

    position_lon = [50-360, 10-360, -10, 5-360, 4-360, 3-360,142-360]
    position_lat = [-40, 40, 48, 78.8, 79.9, 79.1,11]    

    for i in range(len(position_lon)):
        x,y = n(position_lon[i],position_lat[i])
        n.plot(x, y, marker='o', color="yellow")

    plt.savefig('./Output/Fig5_B.jpg')

if __name__== "__main__" :
    main()

KeyboardInterrupt: 